# Retention & Cohort Analysis 

## 1. Cohort Retention

* Q1: Build monthly cohorts by signup date; plot 1-, 3-, 6-month retention curves.

* Q2: Compare retention curves for customers with high vs. low email engagement in their first month.

## 2. RFM Segmentation & Churn

* Q3: Perform RFM segmentation; report purchase frequency and monetary value by segment.

* Q4: Calculate churn rate per RFM segment over 6 months.

## 3. Engagement → Long-Term Value

* Q5: What’s the average lifetime value (LTV) for customers who opened ≥1 email vs. zero?

* Q6: Does early A/B variant engagement predict better long-term retention or LTV?

